In [1]:
import MeCab
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
# MeCabの初期化
mecab = MeCab.Tagger("")

# サンプルの日本語文章
documents = [
    "これはテスト文章です。",
    "もう一つのサンプル文です。",
    "形態素解析を行い、Doc2Vecモデルを作成します。"
]

# 形態素解析を行う関数（特定の品詞を抽出）
def tokenize(text):
    node = mecab.parseToNode(text)
    tokens = []
    while node:
        feature = node.feature.split(',')
        if feature[0] in ['名詞', '動詞', '形容詞']:  # 名詞、動詞、形容詞のみ抽出
            tokens.append(node.surface)
        node = node.next
    return tokens

# ドキュメントを形態素解析し、タグ付きドキュメントとして格納
tagged_data = []
for i, doc in enumerate(documents):
    tokens = tokenize(doc)  # ドキュメントをトークン化
    tagged_document = TaggedDocument(words=tokens, tags=[str(i)])  # トークンとタグを用いてTaggedDocumentを作成
    tagged_data.append(tagged_document)  # tagged_dataリストに追加

# Doc2Vecモデルのトレーニング
model = Doc2Vec(vector_size=100, alpha=0.025, min_alpha=0.00025, min_count=1, dm=1)
model.build_vocab(tagged_data)

# 50エポックでトレーニング
for epoch in range(50):
    print(f'iteration {epoch}')
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    model.alpha -= 0.0002
    model.min_alpha = model.alpha

# モデルの保存
model.save("d2v.model")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49


In [3]:
# モデルのロード
model = Doc2Vec.load("d2v.model")

# 新しいドキュメントの類似度を計算
new_document = "これは新しい文章です。"
new_vector = model.infer_vector(tokenize(new_document))

# 既存のドキュメントと新しいドキュメントの類似度を計算
similarities = []
for i in range(len(documents)):
    doc_vector = model.dv[str(i)]
    similarity = np.dot(new_vector, doc_vector) / (np.linalg.norm(new_vector) * np.linalg.norm(doc_vector))
    similarities.append((i, similarity))

# 類似度の高い順にソート
similarities.sort(key=lambda x: x[1], reverse=True)

# 類似度の結果を表示
for idx, similarity in similarities:
    print(f'Document {idx} Similarity: {similarity:.4f}')

Document 2 Similarity: 0.7878
Document 1 Similarity: 0.7866
Document 0 Similarity: 0.7829
